In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using simpler models with the same results. Can this be considered a victory?

In [3]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..')))
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, ParameterSampler
from imblearn.over_sampling import RandomOverSampler # for oversampling
from sklearn.metrics import classification_report

from utility.classification_utility import *

2025-01-01 23:02:16.683890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735768936.748135  195413 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735768936.767060  195413 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 23:02:16.907981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's start by building our beautiful dataset

In [4]:
cyc = '../dataset/cyclists_cleaned.csv'
races = '../dataset/races_cleaned.csv'
df = make_dataset_for_classification(races, cyc, make_stage_type=True)

100.00%  


In [5]:
# dumb imputations
df['height'] = df['height'].fillna(df['height'].mean())
df['weight'] = df['weight'].fillna(df['weight'].mean())
df['bmi'] = df['weight']/np.square(df['height']/100)
df['cyclist_age_rac'] = df['cyclist_age_rac'].fillna(df['cyclist_age_rac'].mean())
df['climb_total'] = df['climb_total'].fillna(df['length']*0.05) # could be differentiated by lengths
df['steepness'] = df['length'] / df['climb_total']

In [6]:
# for profile, let's use the big stick
# Features and target
# Add a column 'is_ITT' where 1 indicates stage_type is 'ITT', 0 otherwise
df['is_ITT'] = (df['stage_type'] == 'ITT').astype(int)
df_filtered = df[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT', 'profile']].dropna()
X = df_filtered[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT']]
y = df_filtered['profile']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# TODO: test the model
y_pred = clf.predict(X_test)
print(f"Accuracy: {np.mean(y_pred == y_test)}")
print(f"Feature importance: {clf.feature_importances_}")

Accuracy: 0.9991312777770783
Feature importance: [0.21644605 0.27903885 0.30263699 0.20187812 0.        ]


Well.. this might mean that the inferences over the climb total and the length were quite accurate

In [8]:
# Predict missing profiles
missing_profiles = df[df['profile'].isna()]
df.loc[missing_profiles.index, 'profile'] = clf.predict(missing_profiles[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT']])

In [9]:
TO_USE_COLS = [
    # over time
    'total_points',
    'avg_points_per_race', 
    'average_position',
    'avg_speed_cyclist', 
    'mean_stamina_index',
    'race_count',
    # race related
    'length',
    'climb_total', 
    'profile', 
    'startlist_quality', 
    'cyclist_age_rac', 
    'steepness', 
    'is_tarmac', 
    'stage_type',
    # cyclist related
    # 'height',
    # 'weight',
    'bmi',
    'home_game',
    # for the split
    'date',
    'target'
]

df_to_use = df[TO_USE_COLS]

In [10]:
df_tr, df_vl, df_ts = get_data_split(df_to_use)

df_tr = df_tr.drop(columns=['date'])
df_vl = df_vl.drop(columns=['date'])
df_ts = df_ts.drop(columns=['date'])

X_tr, y_tr = split_features_target(df_tr)
X_vl, y_vl = split_features_target(df_vl)
X_ts, y_ts = split_features_target(df_ts)

Normalize the features before predicting

In [11]:
scal = StandardScaler()
X_tr = scal.fit_transform(X_tr)
X_vl = scal.transform(X_vl)
X_ts = scal.transform(X_ts)

Let's build this model

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_tr.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor (validation recall)
    patience=6,            # Number of epochs with no improvement before stopping
    mode='max',            # We want to maximize recall
    restore_best_weights=True  # Restore the model weights from the epoch with the best recall
)


model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[
    'accuracy',
    'recall'
    ])

2025-01-01 21:40:31.233749: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Now fit it

In [12]:
history = model.fit(
    X_tr, y_tr, 
    validation_data=(X_vl, y_vl), 
    epochs=10, 
    batch_size=64,
    verbose=2,
    callbacks=[early_stopping]
)

Epoch 1/10
5550/5550 - 17s - 3ms/step - accuracy: 0.8577 - loss: 0.3597 - recall: 0.0898 - val_accuracy: 0.8626 - val_loss: 0.3374 - val_recall: 0.1284
Epoch 2/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8595 - loss: 0.3535 - recall: 0.1144 - val_accuracy: 0.8648 - val_loss: 0.3342 - val_recall: 0.1900
Epoch 3/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8603 - loss: 0.3520 - recall: 0.1212 - val_accuracy: 0.8635 - val_loss: 0.3354 - val_recall: 0.1388
Epoch 4/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8610 - loss: 0.3508 - recall: 0.1292 - val_accuracy: 0.8634 - val_loss: 0.3361 - val_recall: 0.1961
Epoch 5/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8610 - loss: 0.3500 - recall: 0.1313 - val_accuracy: 0.8646 - val_loss: 0.3323 - val_recall: 0.1765
Epoch 6/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8615 - loss: 0.3491 - recall: 0.1360 - val_accuracy: 0.8642 - val_loss: 0.3338 - val_recall: 0.1788
Epoch 7/10
5550/5550 - 13s - 2ms/step - accuracy: 0.8614 - loss: 0.3484 - recall: 0.1386

In [13]:
val_pred = model.predict(X_vl)
val_pred = (val_pred > 0.5).astype(int)
print(classification_report(y_vl, val_pred))

1359/1359 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
              precision    recall  f1-score   support

       False       0.87      0.99      0.92     37243
        True       0.60      0.13      0.21      6237

    accuracy                           0.86     43480
   macro avg       0.73      0.56      0.57     43480
weighted avg       0.83      0.86      0.82     43480



In [14]:
def build_model(num_layers=2, num_units=64, optimizer='adam', activation='relu', dropout=0.2):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(num_units, activation=activation, input_shape=(X_tr.shape[1],)))
    model.add(tf.keras.layers.Dropout(dropout))
    
    for _ in range(num_layers - 1):
        model.add(tf.keras.layers.Dense(num_units, activation=activation))
        model.add(tf.keras.layers.Dropout(dropout))
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    dict_learning = {
    'adam': tf.keras.optimizers.Adam(),
    'sgd': tf.keras.optimizers.SGD(),
    'lion': tf.keras.optimizers.Lion()
    }
    
    optimizer = dict_learning[optimizer]
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])
    return model

param_grid = {
    'num_layers': [1, 2],
    'num_units': [32, 64],
    'optimizer': ['adam', 'sgd', 'lion'],
    'activation': ['relu', 'tanh'],
    'batch_size': [16, 32, 64],
    'epochs': [5],
}

This proto grid search has already been ran over various parameters. Without verbosity. Won't be uploading on git a long version of the last.

In [19]:
param_list = list(ParameterSampler(param_grid, n_iter=5, random_state=42))

for params in param_list:
    print(f"Training model with parameters: {params}")
    model = build_model(params['num_layers'], params['num_units'], params['optimizer'], params['activation'])
    history = model.fit(
        X_tr, y_tr, 
        validation_data=(X_vl, y_vl), 
        epochs=params['epochs'], 
        batch_size=params['batch_size'],
        verbose=2,
        # callbacks=[early_stopping]
    )
    val_pred = model.predict(X_vl)
    val_pred = (val_pred > 0.5).astype(int)
    # write metrics and params in a file
    with open('model_eval.txt', 'a') as f:
        f.write(f"Parameters: {params}\n")
        f.write(classification_report(y_vl, val_pred))
        f.write('\n\n')
        f.write('\n\n')


Training model with parameters: {'optimizer': 'sgd', 'num_units': 64, 'num_layers': 1, 'epochs': 5, 'batch_size': 16, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22197/22197 - 25s - 1ms/step - accuracy: 0.8564 - loss: 0.3647 - recall: 0.0610 - val_accuracy: 0.8622 - val_loss: 0.3405 - val_recall: 0.1400
Epoch 2/5
22197/22197 - 28s - 1ms/step - accuracy: 0.8584 - loss: 0.3567 - recall: 0.0962 - val_accuracy: 0.8644 - val_loss: 0.3381 - val_recall: 0.1623
Epoch 3/5
22197/22197 - 27s - 1ms/step - accuracy: 0.8587 - loss: 0.3555 - recall: 0.1020 - val_accuracy: 0.8647 - val_loss: 0.3373 - val_recall: 0.1698
Epoch 4/5
22197/22197 - 30s - 1ms/step - accuracy: 0.8589 - loss: 0.3549 - recall: 0.1023 - val_accuracy: 0.8644 - val_loss: 0.3365 - val_recall: 0.1571
Epoch 5/5
22197/22197 - 28s - 1ms/step - accuracy: 0.8588 - loss: 0.3543 - recall: 0.1023 - val_accuracy: 0.8641 - val_loss: 0.3365 - val_recall: 0.1730
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step
Training model with parameters: {'optimizer': 'lion', 'num_units': 32, 'num_layers': 1, 'epochs': 5, 'batch_size': 64, 'activation': 'tanh'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5550/5550 - 7s - 1ms/step - accuracy: 0.8548 - loss: 0.3665 - recall: 0.0956 - val_accuracy: 0.8607 - val_loss: 0.3416 - val_recall: 0.2254
Epoch 2/5
5550/5550 - 7s - 1ms/step - accuracy: 0.8590 - loss: 0.3561 - recall: 0.1081 - val_accuracy: 0.8629 - val_loss: 0.3380 - val_recall: 0.1621
Epoch 3/5
5550/5550 - 7s - 1ms/step - accuracy: 0.8591 - loss: 0.3554 - recall: 0.1125 - val_accuracy: 0.8636 - val_loss: 0.3380 - val_recall: 0.1445
Epoch 4/5
5550/5550 - 7s - 1ms/step - accuracy: 0.8592 - loss: 0.3552 - recall: 0.1139 - val_accuracy: 0.8632 - val_loss: 0.3394 - val_recall: 0.2011
Epoch 5/5
5550/5550 - 6s - 1ms/step - accuracy: 0.8594 - loss: 0.3548 - recall: 0.1159 - val_accuracy: 0.8634 - val_loss: 0.3376 - val_recall: 0.1642
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 644us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 32, 'num_layers': 2, 'epochs': 5, 'batch_size': 32, 'activation': 'relu'}
Epoch 1/5


...

11099/11099 - 17s - 2ms/step - accuracy: 0.8567 - loss: 0.3614 - recall: 0.0931 - val_accuracy: 0.8629 - val_loss: 0.3399 - val_recall: 0.2096
Epoch 2/5
11099/11099 - 15s - 1ms/step - accuracy: 0.8597 - loss: 0.3544 - recall: 0.1159 - val_accuracy: 0.8629 - val_loss: 0.3364 - val_recall: 0.1951
Epoch 3/5
11099/11099 - 16s - 1ms/step - accuracy: 0.8598 - loss: 0.3527 - recall: 0.1203 - val_accuracy: 0.8644 - val_loss: 0.3356 - val_recall: 0.1618
Epoch 4/5
11099/11099 - 15s - 1ms/step - accuracy: 0.8604 - loss: 0.3515 - recall: 0.1252 - val_accuracy: 0.8655 - val_loss: 0.3348 - val_recall: 0.2161
Epoch 5/5
11099/11099 - 14s - 1ms/step - accuracy: 0.8610 - loss: 0.3507 - recall: 0.1291 - val_accuracy: 0.8643 - val_loss: 0.3349 - val_recall: 0.1725
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 32, 'num_layers': 1, 'epochs': 5, 'batch_size': 16, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22197/22197 - 29s - 1ms/step - accuracy: 0.8574 - loss: 0.3600 - recall: 0.0919 - val_accuracy: 0.8633 - val_loss: 0.3383 - val_recall: 0.1760
Epoch 2/5
22197/22197 - 27s - 1ms/step - accuracy: 0.8586 - loss: 0.3557 - recall: 0.1021 - val_accuracy: 0.8640 - val_loss: 0.3367 - val_recall: 0.1764
Epoch 3/5
22197/22197 - 30s - 1ms/step - accuracy: 0.8591 - loss: 0.3547 - recall: 0.1068 - val_accuracy: 0.8632 - val_loss: 0.3382 - val_recall: 0.2131
Epoch 4/5
22197/22197 - 29s - 1ms/step - accuracy: 0.8594 - loss: 0.3540 - recall: 0.1107 - val_accuracy: 0.8640 - val_loss: 0.3368 - val_recall: 0.1669
Epoch 5/5
22197/22197 - 26s - 1ms/step - accuracy: 0.8597 - loss: 0.3535 - recall: 0.1113 - val_accuracy: 0.8632 - val_loss: 0.3357 - val_recall: 0.1772
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 647us/step
Training model with parameters: {'optimizer': 'sgd', 'num_units': 64, 'num_layers': 1, 'epochs': 5, 'batch_size': 64, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5550/5550 - 7s - 1ms/step - accuracy: 0.8550 - loss: 0.3767 - recall: 0.0395 - val_accuracy: 0.8612 - val_loss: 0.3498 - val_recall: 0.0823
Epoch 2/5
5550/5550 - 6s - 1ms/step - accuracy: 0.8571 - loss: 0.3627 - recall: 0.0618 - val_accuracy: 0.8623 - val_loss: 0.3445 - val_recall: 0.1015
Epoch 3/5
5550/5550 - 6s - 1ms/step - accuracy: 0.8575 - loss: 0.3595 - recall: 0.0746 - val_accuracy: 0.8628 - val_loss: 0.3418 - val_recall: 0.1523
Epoch 4/5
5550/5550 - 6s - 1ms/step - accuracy: 0.8580 - loss: 0.3580 - recall: 0.0848 - val_accuracy: 0.8628 - val_loss: 0.3409 - val_recall: 0.1554
Epoch 5/5
5550/5550 - 6s - 1ms/step - accuracy: 0.8583 - loss: 0.3571 - recall: 0.0898 - val_accuracy: 0.8634 - val_loss: 0.3403 - val_recall: 0.1579
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step


Nothing much changes between these different hyperparameters. Maybe the answer lies elsewhere, like in regularization: oversampling and dropout?

In [15]:
param_grid = {
    'dropout': [0.2, 0.5],
    'num_layers': [1, 2],
    'num_units': [32, 64],
    'optimizer': ['adam'],
    'activation': ['relu'],
    'batch_size': [64],
    'epochs': [5],
}

In [16]:
# oversample = RandomOverSampler(sampling_strategy='minority', random_state=42)
# oversampled_X, oversampled_y = oversample.fit_resample(X_tr, y_tr)
# oversampled_X = oversampled_X.reset_index(drop=True)
# oversampled_Y = oversampled_y.reset_index(drop=True)

param_list = list(ParameterSampler(param_grid, n_iter=8, random_state=42))

for params in param_list:
    print(f"Training model with parameters: {params}")
    model = build_model(params['num_layers'], params['num_units'], params['optimizer'], params['activation'])
    history = model.fit(
        oversampled_X, oversampled_y, 
        validation_data=(X_vl, y_vl), 
        epochs=params['epochs'], 
        batch_size=params['batch_size'],
        verbose=2,
        # callbacks=[early_stopping]
    )
    val_pred = model.predict(X_vl)
    val_pred = (val_pred > 0.5).astype(int)
    # write metrics and params in a file
    with open('model_eval.txt', 'a') as f:
        f.write(f"Parameters: {params}\n")
        f.write(classification_report(y_vl, val_pred))
        f.write('\n\n')
        f.write('\n\n')


Training model with parameters: {'optimizer': 'adam', 'num_units': 32, 'num_layers': 1, 'epochs': 5, 'dropout': 0.2, 'batch_size': 64, 'activation': 'relu'}


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-01 23:05:15.535287: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/5
9484/9484 - 13s - 1ms/step - accuracy: 0.6852 - loss: 0.5932 - recall: 0.6917 - val_accuracy: 0.7055 - val_loss: 0.5770 - val_recall: 0.7314
Epoch 2/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6939 - loss: 0.5831 - recall: 0.7026 - val_accuracy: 0.7170 - val_loss: 0.5596 - val_recall: 0.7236
Epoch 3/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6955 - loss: 0.5816 - recall: 0.7055 - val_accuracy: 0.7078 - val_loss: 0.5745 - val_recall: 0.7409
Epoch 4/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6959 - loss: 0.5810 - recall: 0.7052 - val_accuracy: 0.7084 - val_loss: 0.5689 - val_recall: 0.7414
Epoch 5/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6960 - loss: 0.5799 - recall: 0.7058 - val_accuracy: 0.7132 - val_loss: 0.5597 - val_recall: 0.7343
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 64, 'num_layers': 1, 'epochs': 5, 'dropout': 0.2, 'batch_size': 64, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9484/9484 - 13s - 1ms/step - accuracy: 0.6899 - loss: 0.5871 - recall: 0.6920 - val_accuracy: 0.7039 - val_loss: 0.5778 - val_recall: 0.7396
Epoch 2/5
9484/9484 - 13s - 1ms/step - accuracy: 0.6959 - loss: 0.5803 - recall: 0.7027 - val_accuracy: 0.6917 - val_loss: 0.5933 - val_recall: 0.7585
Epoch 3/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6973 - loss: 0.5781 - recall: 0.7050 - val_accuracy: 0.6889 - val_loss: 0.6010 - val_recall: 0.7662
Epoch 4/5
9484/9484 - 11s - 1ms/step - accuracy: 0.6986 - loss: 0.5768 - recall: 0.7052 - val_accuracy: 0.7292 - val_loss: 0.5431 - val_recall: 0.7233
Epoch 5/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6990 - loss: 0.5762 - recall: 0.7058 - val_accuracy: 0.7253 - val_loss: 0.5455 - val_recall: 0.7186
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 553us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 32, 'num_layers': 2, 'epochs': 5, 'dropout': 0.2, 'batch_size': 64, 'activation': 'relu'}
Epoch 1/5


...

9484/9484 - 14s - 1ms/step - accuracy: 0.6895 - loss: 0.5874 - recall: 0.6937 - val_accuracy: 0.7118 - val_loss: 0.5657 - val_recall: 0.7377
Epoch 2/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6964 - loss: 0.5797 - recall: 0.7023 - val_accuracy: 0.7345 - val_loss: 0.5405 - val_recall: 0.7108
Epoch 3/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6981 - loss: 0.5777 - recall: 0.7041 - val_accuracy: 0.6979 - val_loss: 0.5846 - val_recall: 0.7513
Epoch 4/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6987 - loss: 0.5766 - recall: 0.7054 - val_accuracy: 0.7245 - val_loss: 0.5474 - val_recall: 0.7241
Epoch 5/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6988 - loss: 0.5759 - recall: 0.7059 - val_accuracy: 0.7285 - val_loss: 0.5448 - val_recall: 0.7207
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 655us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 32, 'num_layers': 2, 'epochs': 5, 'dropout': 0.5, 'batch_size': 64, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9484/9484 - 14s - 1ms/step - accuracy: 0.6850 - loss: 0.5938 - recall: 0.6869 - val_accuracy: 0.7286 - val_loss: 0.5594 - val_recall: 0.7079
Epoch 2/5
9484/9484 - 13s - 1ms/step - accuracy: 0.6949 - loss: 0.5826 - recall: 0.6922 - val_accuracy: 0.7150 - val_loss: 0.5650 - val_recall: 0.7287
Epoch 3/5
9484/9484 - 13s - 1ms/step - accuracy: 0.6962 - loss: 0.5804 - recall: 0.6954 - val_accuracy: 0.7224 - val_loss: 0.5690 - val_recall: 0.7253
Epoch 4/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6977 - loss: 0.5791 - recall: 0.6943 - val_accuracy: 0.7267 - val_loss: 0.5567 - val_recall: 0.7199
Epoch 5/5
9484/9484 - 12s - 1ms/step - accuracy: 0.6977 - loss: 0.5779 - recall: 0.6948 - val_accuracy: 0.7131 - val_loss: 0.5851 - val_recall: 0.7390
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 615us/step
Training model with parameters: {'optimizer': 'adam', 'num_units': 64, 'num_layers': 2, 'epochs': 5, 'dropout': 0.5, 'batch_size': 64, 'activation': 'relu'}
Epoch 1/5


/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9484/9484 - 15s - 2ms/step - accuracy: 0.6911 - loss: 0.5857 - recall: 0.6911 - val_accuracy: 0.7153 - val_loss: 0.5784 - val_recall: 0.7282
Epoch 2/5
9484/9484 - 13s - 1ms/step - accuracy: 0.6981 - loss: 0.5770 - recall: 0.6972 - val_accuracy: 0.7097 - val_loss: 0.5607 - val_recall: 0.7388
Epoch 3/5
9484/9484 - 14s - 1ms/step - accuracy: 0.7004 - loss: 0.5740 - recall: 0.7002 - val_accuracy: 0.7111 - val_loss: 0.5824 - val_recall: 0.7438
Epoch 4/5
9484/9484 - 14s - 1ms/step - accuracy: 0.7016 - loss: 0.5724 - recall: 0.6989 - val_accuracy: 0.7198 - val_loss: 0.5555 - val_recall: 0.7361
Epoch 5/5
9484/9484 - 13s - 1ms/step - accuracy: 0.7026 - loss: 0.5711 - recall: 0.7006 - val_accuracy: 0.7165 - val_loss: 0.5867 - val_recall: 0.7468
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step
